In [2]:
! ls | grep -i png

edges.png
modelo2.png
modelo.png
out.png
out_semfiltro.png
pdf_to_png.ipynb


In [60]:
# https://docs.opencv.org/3.4/dd/d1a/group__imgproc__feature.html#ga47849c3be0d0406ad3ca45db65a25d2d

In [3]:
# ! conda remove opencv
# ! conda install -c menpo opencv
# ! pip install --upgrade pip
# ! pip install opencv-contrib-python

In [290]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [291]:
img = cv2.imread('samples/5.png',)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
height, width = img.shape

In [292]:
factor = 0.5

height = int(height * factor)
width = int(width * factor)
min_circle_radius = int(20 * factor)
max_circle_radius = int(25 * factor)

In [293]:
# n_height, n_width = int(width/factor), int(height/factor)
img = cv2.resize(img, (width, height))
_, img = cv2.threshold(img, 240, 255, cv2.THRESH_BINARY)
# img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)

In [105]:
# plt.figure(figsize=(10, 12))
# plt.imshow(img, aspect="auto")

cv2.imshow("modelo", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [294]:
# finding lines
import math
import numpy as np

dst = cv2.Canny(img, 300, 400, None, 3)
lines = cv2.HoughLines(
    image=dst, 
    rho=1,
    theta=np.pi/2, 
    threshold=170,
    min_theta=np.pi/2,
    max_theta=np.pi,
)
print(len(lines))

colored = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
if lines is not None:
    for i in range(0, len(lines)):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        a = math.cos(theta)
        b = math.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
        pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
#         if theta >= (75/180.0)*np.pi and theta <= (110/180.0)*np.pi:
#             color = (0, 0, 0)
#             cv2.rectangle(img, pt1, (pt2[0]-50, pt2[1]-50), color, -1)
#             cv2.rectangle(img, pt1, (pt2[0]+50, pt2[1]+50), color, -1)
#         else:
#             color = (0, 0, 0)
#             cv2.rectangle(img, pt1, (pt2[0]-50, pt2[1]-50), color, -1)
        cv2.line(colored, pt1, pt2, (0, 0, 255), 1, cv2.LINE_AA)


cv2.imshow("modelo", colored)
cv2.waitKey(0)
cv2.destroyAllWindows()

22


In [259]:
lines = [line[0][0] for line in lines]
lines = [line for line in lines if line <= height/3]
lines = sorted(lines, reverse=True)
divisor_line = int(lines[0])

In [295]:
divisor_line

205

In [296]:
header = img[:divisor_line, :]
content = img[divisor_line:, :]

In [165]:
circles = cv2.HoughCircles(
    image=content, 
    method=cv2.HOUGH_GRADIENT, 
    dp=1, 
    minDist=int((2*min_circle_radius) + (5*factor)), 
    param1=200, 
    param2=7,
    minRadius=min_circle_radius, 
    maxRadius=max_circle_radius
)
# print(circles)
print(circles.shape)
content.shape

(1, 600, 3)


(964, 827)

In [109]:
min_circle_radius, max_circle_radius

(10, 12)

In [262]:
circles
radii = [c[2] for c in circles[0]]
max(radii), min(radii)

(11.4, 10.2)

In [111]:
img_colored = cv2.cvtColor(content, cv2.COLOR_GRAY2BGR)
if circles is not None:
    circles = np.uint16(np.around(circles))
    for i in circles[0,:]:
        cv2.circle(img_colored, (i[0], i[1]), i[2], (0, 255, 0), cv2.FILLED)
        
cv2.imwrite("out.png", img_colored)

True

In [112]:
# img_colored = cv2.resize(img_colored, (423, 550))
cv2.imshow("modelo", img_colored)
cv2.waitKey(0)
cv2.destroyAllWindows()

## header

In [90]:
# https://docs.opencv.org/2.4/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html
# https://docs.opencv.org/master/d9/d8b/tutorial_py_contours_hierarchy.html

In [297]:
header = img[:divisor_line, :]

In [263]:
cv2.imshow("modelo", header)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [315]:
contours, hierarchy = cv2.findContours(header, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
contours_area = []
# hie = [h for h in hierarchy[0] if h[2] == -1 and h[3] == 0]


header_colored = cv2.cvtColor(header, cv2.COLOR_GRAY2BGR)
for contour in contours:
#     approx = contour
    peri = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.04 * peri, True)
    contours_area.append((cv2.contourArea(approx), approx))
    if contours_area[-1][0] > 300 and contours_area[-1][0] < 1000:
        (x,y,w,h) = cv2.boundingRect(approx)
        cv2.rectangle(header_colored, (x,y), (x+w,y+h), (255,0,0), 2)

In [316]:
cv2.imshow("modelo", header_colored)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [193]:
# hierarchy

In [311]:
len(contours), len(contours_area)

(96, 96)

In [312]:
areas = sorted([a for a, c in contours_area], reverse=True)
contours_area = [ca for ca in contours_area if ca[0] > 600 and ca[0] < 1000]

In [314]:
areas[1:13]

[916.5,
 893.0,
 883.0,
 880.5,
 867.0,
 865.5,
 813.0,
 808.5,
 789.0,
 761.0,
 462.5,
 315.0]

In [279]:
header_colored = cv2.cvtColor(header, cv2.COLOR_GRAY2BGR)
for _, cont in contours_area:
    (x,y,w,h) = cv2.boundingRect(approx)
    cv2.rectangle(header_colored, (x,y), (x+w,y+h), (255,0,0), 2)

In [221]:
sorted(contours_area, reverse=True)[1:14]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [137]:
mode = max(set(contours_area), key=contours_area.count) # achando area moda

boxes = [contour for contour, area in zip(contours, contours_area) if area == mode]
mode

0.0

In [268]:
header_colored = cv2.cvtColor(header, cv2.COLOR_GRAY2BGR)
for box in boxes:
    (x,y,w,h) = cv2.boundingRect(box)
    cv2.rectangle(header_colored, (x,y), (x+w,y+h), (0,0,255), 2)

In [299]:
cv2.imshow("modelo", header_colored)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [236]:
from sklearn.cluster import KMeans

areas = np.array(contours_area).reshape(-1, 1)
# areas

In [247]:
np.set_printoptions(suppress=True)
kmeans = KMeans(n_clusters=10, random_state=0).fit(areas)

kmeans.cluster_centers_, kmeans.labels_

(array([[ 13773.        ],
        [158845.        ],
        [   151.86363636],
        [  1025.        ],
        [  3708.        ],
        [    16.17857143],
        [   335.75      ],
        [   100.16666667],
        [   206.        ],
        [   948.        ]]),
 array([5, 5, 5, 5, 7, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 9, 5, 5, 5,
        7, 7, 7, 7, 5, 5, 5, 2, 5, 5, 7, 5, 2, 2, 2, 2, 2, 6, 8, 2, 8, 8,
        8, 6, 6, 2, 2, 6, 2, 5, 4, 1], dtype=int32))

In [248]:
keys = set(kmeans.labels_)
rep = {k: 0 for k in keys}

for lbl in kmeans.labels_:
    rep[lbl] = rep[lbl] + 1
    
rep

{0: 1, 1: 1, 2: 11, 3: 11, 4: 1, 5: 14, 6: 4, 7: 6, 8: 4, 9: 1}